In [1]:
import numpy as np
import os, pickle, sys, json, random, math 
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
# from keras.np_utils import probas_to_classes
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, add, concatenate, TimeDistributed, Bidirectional, merge
from sklearn.metrics.pairwise import cosine_similarity
from keras.layers import Dense, Input, Flatten, Merge, Dropout, concatenate, Concatenate
from sklearn.metrics import accuracy_score, f1_score

/home/gauravroy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
raw_train = []
with open('/home/gauravroy/Desktop/slotfilling/ATIS_train.iob','r') as f:
    for line in f:
        raw_train.append(line)
        
raw_test = []
with open('/home/gauravroy/Desktop/slotfilling/ATIS_test.iob','r') as f:
    for line in f:
        raw_test.append(line)

with open('/home/gauravroy/Desktop/slotfilling/embedding_matrix.pickle', 'rb') as fp:
    embedding_matrix = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/embedding_matrix_w.pickle', 'rb') as fp:
    embedding_matrix_w = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/embedding_matrix_g.pickle', 'rb') as fp:
    embedding_matrix_g = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/embedding_matrix_pos.pickle', 'rb') as fp:
     embedding_matrix_pos = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/train_sequence.pickle', 'rb') as fp:
    train_sequence = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_sequence.pickle', 'rb') as fp:
    test_sequence = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/train_sequence1.pickle', 'rb') as fp:
    train_sequence1 = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/test_sequence1.pickle', 'rb') as fp:
    test_sequence1 = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling//slot_word_dict.pickle', 'rb') as fp:
    slot_word_dict = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/y_train_slot.pickle', 'rb') as fp:
    y_train_slot = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/y_test_slot.pickle', 'rb') as fp:
    y_test_slot = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/x_train_pos.pickle', 'rb') as fp:
    x_train_pos = pickle.load(fp)
fp.close()

with open('/home/gauravroy/Desktop/slotfilling/x_test_pos.pickle', 'rb') as fp:
    x_test_pos = pickle.load(fp)
fp.close()

slot_dict, words_dict = slot_word_dict

train_sequence = pad_sequences(train_sequence, maxlen = 46, padding='post')
test_sequence = pad_sequences(test_sequence, maxlen = 46, padding='post') 

max_len = 46

In [3]:
n = len(slot_dict)

y_train_slot = list()
for utterance in raw_train:
    outs = np.zeros((max_len,len(slot_dict)))
    outs = list(outs)
    slots = utterance.split('\t')[1].strip().split(' ')[1:-1]
    for i in range(len(slots)):
        outs[i][slot_dict[slots[i]]] = 1
    y_train_slot.append(outs)
    
y_test_slot = list()
for utterance in raw_test:
    outs = np.zeros((max_len,len(slot_dict)))
    outs = list(outs)
    slots = utterance.split('\t')[1].strip().split(' ')[1:-1]
    for i in range(len(slots)):
        outs[i][slot_dict[slots[i]]] = 1
    y_test_slot.append(outs)

In [4]:
n = len(slot_dict)

z_train_slot = list()
for utterance in raw_train:
    outs = list()
    slots = utterance.split('\t')[1].strip().split(' ')[1:-1]
    for i in range(len(slots)):
        outs.append(slot_dict[slots[i]])
    z_train_slot.append(outs)
    
z_test_slot = list()
for utterance in raw_test:
    outs = list()
    slots = utterance.split('\t')[1].strip().split(' ')[1:-1]
    for i in range(len(slots)):
        outs.append(slot_dict[slots[i]])
    z_test_slot.append(outs)

In [5]:
def flatten(mat):
    arr = list()
    for ar in mat:
        for a in ar:
            arr.append(a)
    return arr

In [6]:
def make_z_pred_slot(mat,z_act_slot):
    ans = list()
    for i in range(len(mat)):
        ans.append(mat[i][:len(z_act_slot[i])])
    return ans

In [7]:
def accu(list1, list2):
    if(len(list1) != len(list2)):
        print("Size of a the lists not equal")
        return
    count = 0.0
    for i in range(len(list1)):
        if(list1[i] == list2[i]):
            count = count + 1
            
    return count/len(list1)  

In [8]:
lstm_embedding_layer = Embedding(len(embedding_matrix_g), len(embedding_matrix_g[0]), weights=[embedding_matrix_g], input_length=max_len, 
                            trainable=False, mask_zero = True)

sequence_input = Input(shape=(max_len,), dtype='int32')

lstm_embedded_sequences = lstm_embedding_layer(sequence_input)

forward = LSTM(200, return_sequences=True)(lstm_embedded_sequences)
backward = LSTM(200, return_sequences=True,go_backwards=True)(lstm_embedded_sequences)

tagger = merge([forward, backward], mode='concat')

#tagger = Dropout(self.dropout_ratio)(tagger)

prediction = TimeDistributed(Dense(output_dim=127, activation='softmax'))(tagger)

out = Dense(units=200, activation='relu', kernel_initializer='he_normal')(prediction)
out = Dense(units=len(slot_dict), activation='softmax', kernel_initializer='he_normal')(out)

graph = Model(inputs=sequence_input, outputs=prediction)
graph.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy','categorical_accuracy'])

/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()
/home/gauravroy/.local/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/gauravroy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=127)`
  from ipykernel import kernelapp as app


In [9]:
saveBestModel = ModelCheckpoint(filepath="/home/gauravroy/Desktop/slotfilling/hdf5/g_lstm.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

graph.fit(train_sequence, np.array(y_train_slot),validation_data=(test_sequence,np.array(y_test_slot)), epochs=10, batch_size=50, verbose=2, callbacks=[saveBestModel])
#graph.load_weights("/home/gauravroy/Desktop/slotfilling/hdf5/g_lstm.hdf5")

lstm_pred_train = graph.predict(train_sequence)
lstm_pred_test = graph.predict(test_sequence)

lstm_pred_classes_train = lstm_pred_train.argmax(axis=-1)
lstm_pred_classes_test = lstm_pred_test.argmax(axis=-1)

Train on 4978 samples, validate on 893 samples
Epoch 1/10
 - 34s - loss: 2.9899 - acc: 0.6348 - categorical_accuracy: 0.6348 - val_loss: 2.4942 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00001: val_acc improved from -inf to 0.59935, saving model to /home/gauravroy/Desktop/slotfilling/hdf5/g_lstm.hdf5


/home/gauravroy/.local/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer merge_1 was passed non-serializable keyword arguments: {'mask': [<tf.Tensor 'embedding_1/NotEqual:0' shape=(?, 46) dtype=bool>, <tf.Tensor 'embedding_1/NotEqual:0' shape=(?, 46) dtype=bool>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/10
 - 33s - loss: 2.1198 - acc: 0.6348 - categorical_accuracy: 0.6348 - val_loss: 2.2033 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00002: val_acc did not improve from 0.59935
Epoch 3/10
 - 35s - loss: 1.9281 - acc: 0.6347 - categorical_accuracy: 0.6347 - val_loss: 2.0718 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00003: val_acc did not improve from 0.59935
Epoch 4/10
 - 35s - loss: 1.8484 - acc: 0.6345 - categorical_accuracy: 0.6345 - val_loss: 2.0275 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00004: val_acc did not improve from 0.59935
Epoch 5/10
 - 36s - loss: 1.7934 - acc: 0.6347 - categorical_accuracy: 0.6347 - val_loss: 1.9655 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00005: val_acc did not improve from 0.59935
Epoch 6/10
 - 36s - loss: 1.7367 - acc: 0.6347 - categorical_accuracy: 0.6347 - val_loss: 1.9137 - val_acc: 0.5994 - val_categorical_accuracy: 0.5994

Epoch 00006: val_acc did not improve from 0.5

In [10]:
z_pred_slot_train = make_z_pred_slot(lstm_pred_classes_train,z_train_slot)
z_pred_slot_test = make_z_pred_slot(lstm_pred_classes_test,z_test_slot)

In [11]:
print ("Train F1: " + str(f1_score(flatten(z_train_slot),flatten(z_pred_slot_train), average = 'weighted')))
print ("Test F1: " + str(f1_score(flatten(z_test_slot),flatten(z_pred_slot_test), average = 'weighted')))

Train F1: 0.49284069196626834
Test F1: 0.4503458217073495


/home/gauravroy/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
print ("Train Accuracy: " + str(accu(flatten(z_train_slot),flatten(z_pred_slot_train))*100) + "%")
print ("Test Accuracy: " + str(accu(flatten(z_test_slot),flatten(z_pred_slot_test))*100) + "%")

Train Accuracy: 63.46797153024911%
Test Accuracy: 60.028371890004365%
